#### Cleaning up the official landprice data
国土数値情報ダウンロードサイト<br>
https://nlftp.mlit.go.jp/ksj/old/datalist/old_KsjTmplt-L01.html<br>
TOP>国土数値情報>旧統一フォーマットテキスト形式>地価公示（ポイント）>「全国」データのダウンロード>令和５年<br>
※オリジナルデータのタイトルは「L01-2023P-2K.csv」

In [1]:
import pandas as pd
from decimal import Decimal, ROUND_DOWN
import numpy as np

In [2]:
file = "L01-2023P-2K.csv"
df = pd.read_csv(file)

#### delete unneeded colums

In [3]:
colums_to_be_deleted = """
所在地コード
連番
年次
前年所在地コード
前年用途
前年連番
市区町村名
住居表示
行政
地積
利用の現況
利用状況表示
利用区分
建物構造
施設
形状区分
間口（比率）
奥行（比率）
階層（地上）
階層（地下）
前面道路区分
前面道路の方位区分
前面道路の幅員
前面道路の駅前区分
前面道路の舗装状況
側道区分
側道方位区分
交通施設との近接区分
用途区分
防火区分
都市計画区分
高度地区
森林区分
公園区分
建蔽率
容積率
割増容積率
共通地点区分
対前年変動率
選定年次ビット
属性移動Ｓ５９
属性移動Ｓ６０
属性移動Ｓ６１
属性移動Ｓ６２
属性移動Ｓ６３
属性移動Ｈ１
属性移動Ｈ２
属性移動Ｈ３
属性移動Ｈ４
属性移動Ｈ５
属性移動Ｈ６
属性移動Ｈ７
属性移動Ｈ８
属性移動Ｈ９
属性移動Ｈ１０
属性移動Ｈ１１
属性移動Ｈ１２
属性移動Ｈ１３
属性移動Ｈ１４
属性移動Ｈ１５
属性移動Ｈ１６
属性移動Ｈ１７
属性移動Ｈ１８
属性移動Ｈ１９
属性移動Ｈ２０
属性移動Ｈ２１
属性移動Ｈ２２
属性移動Ｈ２３
属性移動Ｈ２４
属性移動Ｈ２５
属性移動Ｈ２６
属性移動Ｈ２７
属性移動Ｈ２８
属性移動Ｈ２９
属性移動Ｈ３０
属性移動Ｈ３１
属性移動Ｒ２
属性移動Ｒ３
属性移動Ｒ４
属性移動Ｒ５
"""
colums_to_be_deleted = colums_to_be_deleted.strip().split('\n')
df1 = df.drop(columns=colums_to_be_deleted)

#### Rename columns

In [4]:
renamelist={"経度":"longitude",
            "緯度":"latitude",
            "用途":"usage",
            "所在並びに地番":"address",
            "周辺の土地の利用の現況":"surroundings",
            "駅名":"station",
            "Ｓ５８価格":"price1983",
            "Ｓ５９価格":"price1984",
            "Ｓ６０価格":"price1985",
            "Ｓ６１価格":"price1986",
            "Ｓ６２価格":"price1987",
            "Ｓ６３価格":"price1988",
            "Ｈ１価格":"price1989",
            "Ｈ２価格":"price1990",
            "Ｈ３価格":"price1991",
            "Ｈ４価格":"price1992",
            "Ｈ５価格":"price1993",
            "Ｈ６価格":"price1994",
            "Ｈ７価格":"price1995",
            "Ｈ８価格":"price1996",
            "Ｈ９価格":"price1997",
            "Ｈ１０価格":"price1998",
            "Ｈ１１価格":"price1999",
            "Ｈ１２価格":"price2000",
            "Ｈ１３価格":"price2001",
            "Ｈ１４価格":"price2002",
            "Ｈ１５価格":"price2003",
            "Ｈ１６価格":"price2004",
            "Ｈ１７価格":"price2005",
            "Ｈ１８価格":"price2006",
            "Ｈ１９価格":"price2007",
            "Ｈ２０価格":"price2008",
            "Ｈ２１価格":"price2009",
            "Ｈ２２価格":"price2010",
            "Ｈ２３価格":"price2011",
            "Ｈ２４価格":"price2012",
            "Ｈ２５価格":"price2013",
            "Ｈ２６価格":"price2014",
            "Ｈ２７価格":"price2015",
            "Ｈ２８価格":"price2016",
            "Ｈ２９価格":"price2017",
            "Ｈ３０価格":"price2018",
            "Ｈ３１価格":"price2019",
            "Ｒ２価格":"price2020",
            "Ｒ３価格":"price2021",
            "Ｒ４価格":"price2022",
            "Ｒ５価格":"price2023"}
df2 = df1.rename(columns=renamelist)

#### divide coordinates by 3600

In [5]:
df2["longitude"] = df2["longitude"] / 3600
df2["latitude"] = df2["latitude"] / 3600

#### Round down coordinates at 7th digits

In [6]:
def quantize_decimal(value):
    return Decimal(value).quantize(Decimal('0.000001'), rounding=ROUND_DOWN)

df2['longitude'] = df2['longitude'].apply(quantize_decimal)
df2['latitude'] = df2['latitude'].apply(quantize_decimal)

#### Replace usage<br>0:住宅地→１<br>3:宅地見込地→４<br>5:商業地→２<br>7:準工業地→４<br>9:工業地→３<br>10:市街化調整区域内の現況宅地→４<br>13:市街化調整区域内の現況林地→４<br>変換後は住宅地を１、商業地を２、工業地を３、それ以外は４と表示

In [7]:
replace_usage = {0: 1, 3: 4, 5: 2, 7: 4, 9: 3, 10: 4, 13: 4}
df2["usage"] = df2["usage"].replace(replace_usage)

#### Split address into pref and address by space

In [8]:
df2['pref'], df2['address'] = df2['address'].str.split('　', 1).str

C:\Users\ukoda\AppData\Local\Temp\ipykernel_7412\3819294185.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df2['pref'], df2['address'] = df2['address'].str.split('　', 1).str
C:\Users\ukoda\AppData\Local\Temp\ipykernel_7412\3819294185.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df2['pref'], df2['address'] = df2['address'].str.split('　', 1).str


In [9]:
df2["駅距離"] = np.where((df2["駅距離"] / 1000) % 1 == 0,
                         (df2["駅距離"] / 1000).astype(int).astype(str),
                         (df2["駅距離"] / 1000).astype(str)) + "km"
df2["station"] = df2["station"] + "まで"
df2['station'] = df2['station'] + df2['駅距離']
df2 = df2.drop(columns='駅距離')

#### Change pref into codes<br>nlftp.mlit.go.jp/ksj/gml/codelist/PrefCd.html

In [10]:
pref_code = {
    "北海道": "01", "青森県": "02", "岩手県": "03", "宮城県": "04",
    "秋田県": "05", "山形県": "06", "福島県": "07", "茨城県": "08",
    "栃木県": "09", "群馬県": "10", "埼玉県": "11", "千葉県": "12",
    "東京都": "13", "神奈川県": "14", "新潟県": "15", "富山県": "16",
    "石川県": "17", "福井県": "18", "山梨県": "19", "長野県": "20",
    "岐阜県": "21", "静岡県": "22", "愛知県": "23", "三重県": "24",
    "滋賀県": "25", "京都府": "26", "大阪府": "27", "兵庫県": "28",
    "奈良県": "29", "和歌山県": "30", "鳥取県": "31", "島根県": "32",
    "岡山県": "33", "広島県": "34", "山口県": "35", "徳島県": "36",
    "香川県": "37", "愛媛県": "38", "高知県": "39", "福岡県": "40",
    "佐賀県": "41", "長崎県": "42", "熊本県": "43", "大分県": "44",
    "宮崎県": "45", "鹿児島県": "46", "沖縄県": "47"}
df2["pref"] = df2["pref"].replace(pref_code)

#### divide prices by 10000

In [11]:
prices = """
price1983
price1984
price1985
price1986
price1987
price1988
price1989
price1990
price1991
price1992
price1993
price1994
price1995
price1996
price1997
price1998
price1999
price2000
price2001
price2002
price2003
price2004
price2005
price2006
price2007
price2008
price2009
price2010
price2011
price2012
price2013
price2014
price2015
price2016
price2017
price2018
price2019
price2020
price2021
price2022
price2023
"""
prices = prices.strip().split('\n')
df2[prices] = df2[prices] / 10000

#### replace 0 by blank

In [12]:
df2[prices] = df2[prices].replace({0: ""})

#### Rearrange columns order

In [13]:
columns_order = """
longitude
latitude
usage
pref
address
surroundings
station
price1983
price1984
price1985
price1986
price1987
price1988
price1989
price1990
price1991
price1992
price1993
price1994
price1995
price1996
price1997
price1998
price1999
price2000
price2001
price2002
price2003
price2004
price2005
price2006
price2007
price2008
price2009
price2010
price2011
price2012
price2013
price2014
price2015
price2016
price2017
price2018
price2019
price2020
price2021
price2022
price2023
"""
columns_order = columns_order.strip().split("\n")
df2 = df2[columns_order]

#### Integrate various "-" and "ー"s

In [14]:
df2['surroundings'] = df2['surroundings'].str.replace('-', 'ー')  # 半角ハイフン
df2['surroundings'] = df2['surroundings'].str.replace('−', 'ー')  # 全角ダッシュ
df2['surroundings'] = df2['surroundings'].str.replace('－', 'ー')  # 全角ハイフン
df2['surroundings'] = df2['surroundings'].str.replace('—', 'ー')  # emダッシュ
df2['surroundings'] = df2['surroundings'].str.replace('―', 'ー')  # enダッシュ
df2['address'] = df2['address'].str.replace('-', 'ー')  # 半角ハイフン
df2['address'] = df2['address'].str.replace('−', 'ー')  # 全角ダッシュ
df2['address'] = df2['address'].str.replace('－', 'ー')  # 全角ハイフン
df2['address'] = df2['address'].str.replace('—', 'ー')  # emダッシュ
df2['address'] = df2['address'].str.replace('―', 'ー')  # enダッシュ

In [15]:
df2.to_csv("dataset_ver4.csv", index=False)